# Insights 

---

## Library Imports

In [ ]:
# Core data manipulation
import pandas as pd
import numpy as np
import json
from pathlib import Path

# Statistics
from scipy import stats
from scipy.stats import (
    ttest_ind, f_oneway, chi2_contingency, 
    pearsonr, spearmanr, shapiro, levene
)

# Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import (
    r2_score, mean_squared_error, classification_report,
    confusion_matrix, roc_auc_score, silhouette_score
)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully")

---

## Load Data

In [ ]:
# Load the enhanced reviews dataset
data_path = '/Users/jamesroot/Desktop/JAMES/Noetheca/Reviews/Data/reviews_enhanced.csv'
df = pd.read_csv(data_path)

print(f"✅ Data loaded successfully")
print(f"Shape: {df.shape}")
print(f"Columns: {len(df.columns)}")

---

## Data Collection and Overview

In [ ]:
# Basic dataset info
print("="*80)
print("DATASET OVERVIEW")
print("="*80)
print(f"\nTotal reviews: {len(df):,}")
print(f"Total features: {len(df.columns)}")
print(f"Movies: {df['Movie_Title'].nunique()}")

# Convert Review_Date to datetime if it's a string
if df['Review_Date'].dtype == 'object':
    df['Review_Date'] = pd.to_datetime(df['Review_Date'], errors='coerce')
    
print(f"Date range: {df['Review_Date'].min()} to {df['Review_Date'].max()}")

print("\n" + "="*80)
print("MISSING DATA SUMMARY")
print("="*80)
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing_Count': missing[missing > 0],
    'Missing_Pct': missing_pct[missing > 0]
}).sort_values('Missing_Count', ascending=False)

if len(missing_df) > 0:
    print(missing_df)
else:
    print("No missing data found!")

print("\n" + "="*80)
print("DATA TYPES")
print("="*80)
print(df.dtypes.value_counts())

---

## Classify features by type

In [ ]:
# Classify features by type
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

# Remove ID and non-analytic columns
exclude_cols = [
    'Review_ID', 'Review_Text', 'Reviewer', 'Review_Date', 'Movie_Title', 
    'Review_Title', 'Source', 'username_gender_hint', 'username_age_hint',
    'username_interests', 'username_patterns', 'movies_mentioned', 
    'comparison_context', 'love_statements', 'hate_statements', 
    'wish_statements', 'questions', 'review_window'
]
numeric_features = [col for col in numeric_features if col not in exclude_cols]
categorical_features = [col for col in categorical_features if col not in exclude_cols]

print("="*80)
print("FEATURE CLASSIFICATION")
print("="*80)
print(f"\nNumeric features ({len(numeric_features)}):")
for i in range(0, len(numeric_features), 5):
    print(numeric_features[i:i+5])

print(f"\nCategorical features ({len(categorical_features)}):")
print(categorical_features)

# Store for later use
print(f"\n✅ Identified {len(numeric_features)} numeric and {len(categorical_features)} categorical features")

---

## Target Variables

In [ ]:
# Define key outcome variables for analysis
print("="*80)
print("TARGET VARIABLES FOR ANALYSIS")
print("="*80)

# Primary outcome: Rating
print(f"\nRating distribution:")
print(df['Rating'].describe())

# Create binary outcome: High performer (>7.0)
df['high_performer'] = (df['Rating'] > 7.0).astype(int)
print(f"\nHigh performers (>7.0): {df['high_performer'].sum()} ({df['high_performer'].mean()*100:.1f}%)")

# Create polarization indicator
df['is_polarizing'] = ((df['love_count'] > 0) & (df['hate_count'] > 0)).astype(int)
print(f"Polarizing reviews: {df['is_polarizing'].sum()} ({df['is_polarizing'].mean()*100:.1f}%)")

print("\n✅ Target variables defined")

---

## Movie Level Aggregation 

In [ ]:
# Create movie-level summary statistics for cross-movie analysis
movie_stats = df.groupby('Movie_Title').agg({
    'Rating': ['mean', 'std', 'count'],
    'love_count': 'mean',
    'hate_count': 'mean',
    'vader_compound': 'mean',
    'total_votes': 'sum',
    'high_performer': 'mean',
    'is_polarizing': 'mean'
}).round(3)

# Flatten column names
movie_stats.columns = ['_'.join(col).strip() for col in movie_stats.columns.values]
movie_stats = movie_stats.reset_index()

print("="*80)
print("MOVIE-LEVEL SUMMARY STATISTICS")
print("="*80)
print(movie_stats.to_string())

print("\n✅ Movie-level data aggregated")

---

## Correlation Matrix Predictors

In [ ]:
# Correlation analysis: Which features correlate with Rating?
print("="*80)
print("CORRELATION ANALYSIS: RATING PREDICTORS")
print("="*80)

# Calculate correlations with Rating
correlations = df[numeric_features].corr()['Rating'].sort_values(ascending=False)

# Remove Rating's self-correlation
correlations = correlations.drop('Rating')

print("\nTop 20 Positive Correlations with Rating:")
print(correlations.head(20))

print("\nTop 20 Negative Correlations with Rating:")
print(correlations.tail(20))

# Statistical significance test for top correlations
print("\n" + "="*80)
print("STATISTICAL SIGNIFICANCE OF TOP CORRELATIONS")
print("="*80)

top_features = list(correlations.head(10).index) + list(correlations.tail(10).index)

for feature in top_features:
    r, p_value = pearsonr(df[feature].dropna(), df.loc[df[feature].notna(), 'Rating'])
    significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
    print(f"{feature:30s} r={r:6.3f}, p={p_value:.4e} {significance}")

print("\n✅ Correlation analysis complete")

---

## Visualize top correlations

In [ ]:
# Visualize top correlations
print("="*80)
print("VISUALIZATION: TOP 15 RATING CORRELATIONS")
print("="*80)

# Get top 15 positive and negative (exclude NaN)
correlations_clean = correlations.dropna()
top_pos = correlations_clean.head(15)
top_neg = correlations_clean.tail(15)
top_combined = pd.concat([top_pos, top_neg])

# Create horizontal bar plot
plt.figure(figsize=(10, 8))
colors = ['green' if x > 0 else 'red' for x in top_combined.values]
plt.barh(range(len(top_combined)), top_combined.values, color=colors, alpha=0.6)
plt.yticks(range(len(top_combined)), top_combined.index, fontsize=9)
plt.xlabel('Correlation with Rating', fontsize=11)
plt.title('Top Features Correlated with Rating', fontsize=13, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("✅ Visualization complete")

---

## Linear Regression to Predict Rating

In [ ]:
# Multiple linear regression to predict Rating
print("="*80)
print("LINEAR REGRESSION: PREDICTING RATING FROM ALL FEATURES")
print("="*80)

# Prepare features - EXCLUDE Rating and derived variables
features_to_exclude = ['Rating', 'high_performer', 'is_polarizing']
predictor_features = [f for f in numeric_features if f not in features_to_exclude]

print(f"Using {len(predictor_features)} predictor features (excluded: {features_to_exclude})")

# Prepare data - remove rows with missing values
X = df[predictor_features].dropna()
y = df.loc[X.index, 'Rating']

print(f"Sample size: n={len(X):,} reviews")

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predictions
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

# Model performance
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"\nModel Performance:")
print(f"  Training R² = {train_r2:.4f}")
print(f"  Test R² = {test_r2:.4f}")
print(f"  Training RMSE = {train_rmse:.4f}")
print(f"  Test RMSE = {test_rmse:.4f}")

# Feature importance (coefficients)
feature_importance = pd.DataFrame({
    'Feature': predictor_features,
    'Coefficient': lr.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print("\nTop 20 Most Important Features (by |coefficient|):")
print(feature_importance.head(20).to_string(index=False))

print("\n✅ Linear regression complete")

---

## Ridge Regression to handle multicollinearity

In [ ]:
# Ridge regression to handle multicollinearity
print("="*80)
print("RIDGE REGRESSION: REGULARIZED PREDICTION")
print("="*80)

# Use the same predictor features (excluding Rating)
X = df[predictor_features].dropna()
y = df.loc[X.index, 'Rating']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Ridge regression with cross-validation
alphas = [0.001, 0.01, 0.1, 1, 10, 100]
ridge_scores = []

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    scores = cross_val_score(ridge, X_scaled, y, cv=5, scoring='r2')
    ridge_scores.append(scores.mean())
    print(f"Alpha={alpha:6.3f}: Mean R² = {scores.mean():.4f} (±{scores.std():.4f})")

# Best alpha
best_alpha = alphas[np.argmax(ridge_scores)]
print(f"\nBest alpha: {best_alpha}")

# Fit final model with train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
ridge_final = Ridge(alpha=best_alpha)
ridge_final.fit(X_train, y_train)

y_pred_ridge = ridge_final.predict(X_test)
ridge_r2 = r2_score(y_test, y_pred_ridge)
ridge_rmse = np.sqrt(mean_squared_error(y_test, y_pred_ridge))

print(f"\nFinal Ridge Model Performance:")
print(f"  Test R² = {ridge_r2:.4f}")
print(f"  Test RMSE = {ridge_rmse:.4f}")

print("\n✅ Ridge regression complete")

---

## K-Means clustering

In [ ]:
# K-Means clustering to identify audience segments
print("="*80)
print("K-MEANS CLUSTERING: AUDIENCE SEGMENTATION")
print("="*80)

# Select features for clustering (sentiment, emotions, engagement)
cluster_features = [
    'vader_compound', 'vader_pos', 'vader_neg',
    'emotion_joy', 'emotion_trust', 'emotion_fear', 'emotion_surprise',
    'emotion_sadness', 'emotion_disgust', 'emotion_anger', 'emotion_anticipation',
    'love_count', 'hate_count', 'wish_count',
    'total_votes', 'helpfulness_ratio',
    'flesch_reading_ease', 'exclamation_count'
]

# Prepare data
X_cluster = df[cluster_features].dropna()
print(f"Sample size: n={len(X_cluster):,} reviews")
print(f"Clustering on {len(cluster_features)} features")

# Standardize
scaler_cluster = StandardScaler()
X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)

# Test different numbers of clusters
inertias = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_cluster_scaled)
    inertias.append(kmeans.inertia_)
    sil_score = silhouette_score(X_cluster_scaled, kmeans.labels_)
    silhouette_scores.append(sil_score)
    print(f"k={k}: Inertia={kmeans.inertia_:.2f}, Silhouette={sil_score:.4f}")

print("\n✅ Clustering analysis complete")

---

## Fit Optimal Clustering

In [ ]:
# Fit optimal clustering solution (k=3)
print("="*80)
print("3-CLUSTER SOLUTION: AUDIENCE SEGMENT PROFILES")
print("="*80)

kmeans_final = KMeans(n_clusters=3, random_state=42, n_init=10)
cluster_labels = kmeans_final.fit_predict(X_cluster_scaled)

# Add cluster labels to dataframe
df_clustered = df.loc[X_cluster.index].copy()
df_clustered['cluster'] = cluster_labels

print(f"Cluster sizes:")
print(df_clustered['cluster'].value_counts().sort_index())

# Profile each cluster
print("\n" + "="*80)
print("CLUSTER PROFILES")
print("="*80)

for cluster_id in range(3):
    cluster_data = df_clustered[df_clustered['cluster'] == cluster_id]
    
    print(f"\n{'='*80}")
    print(f"CLUSTER {cluster_id} (n={len(cluster_data)})")
    print(f"{'='*80}")
    
    # Key statistics
    print(f"\nRating: {cluster_data['Rating'].mean():.2f} (±{cluster_data['Rating'].std():.2f})")
    print(f"Sentiment (vader_compound): {cluster_data['vader_compound'].mean():.3f}")
    print(f"Love count: {cluster_data['love_count'].mean():.2f}")
    print(f"Hate count: {cluster_data['hate_count'].mean():.2f}")
    print(f"Total votes: {cluster_data['total_votes'].mean():.1f}")
    
    # Emotions
    print(f"\nTop emotions:")
    emotion_cols = ['emotion_joy', 'emotion_trust', 'emotion_fear', 'emotion_surprise',
                    'emotion_sadness', 'emotion_disgust', 'emotion_anger', 'emotion_anticipation']
    emotion_means = cluster_data[emotion_cols].mean().sort_values(ascending=False)
    for emotion, value in emotion_means.head(3).items():
        print(f"  {emotion}: {value:.3f}")
    
    # Gender distribution
    if 'username_gender_hint' in cluster_data.columns:
        gender_dist = cluster_data['username_gender_hint'].value_counts(normalize=True)
        print(f"\nGender distribution:")
        for gender, pct in gender_dist.items():
            if gender in ['male', 'female']:
                print(f"  {gender}: {pct*100:.1f}%")
    
    # High performers
    if 'high_performer' in cluster_data.columns:
        high_perf_pct = cluster_data['high_performer'].mean() * 100
        print(f"\nHigh performers (>7.0): {high_perf_pct:.1f}%")

print("\n✅ Cluster profiling complete")

---

## Principle Component Analysis

In [ ]:
# Principal Component Analysis
print("="*80)
print("PRINCIPAL COMPONENT ANALYSIS (PCA)")
print("="*80)

# Use all numeric predictors
X_pca = df[predictor_features].dropna()
print(f"Sample size: n={len(X_pca):,}")

# Standardize
scaler_pca = StandardScaler()
X_pca_scaled = scaler_pca.fit_transform(X_pca)

# Fit PCA
pca = PCA()
pca.fit(X_pca_scaled)

# Variance explained
variance_explained = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(variance_explained)

print(f"\nVariance explained by first 10 components:")
for i in range(10):
    print(f"  PC{i+1}: {variance_explained[i]*100:.2f}% (cumulative: {cumulative_variance[i]*100:.2f}%)")

# How many components for 80% variance?
n_components_80 = np.argmax(cumulative_variance >= 0.80) + 1
print(f"\nComponents needed for 80% variance: {n_components_80}")

# Scree plot
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(1, 21), variance_explained[:20], 'bo-')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.title('Scree Plot')
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(range(1, 21), cumulative_variance[:20], 'ro-')
plt.axhline(y=0.80, color='g', linestyle='--', label='80% threshold')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Variance Explained')
plt.title('Cumulative Variance')
plt.legend()
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ PCA complete")

In [ ]:
# Interpret the top principal components
print("="*80)
print("INTERPRETING TOP PRINCIPAL COMPONENTS")
print("="*80)

# Refit PCA with first 5 components for interpretation
pca_5 = PCA(n_components=5)
pca_5.fit(X_pca_scaled)

# Get loadings
loadings = pd.DataFrame(
    pca_5.components_.T,
    columns=[f'PC{i+1}' for i in range(5)],
    index=predictor_features
)

# Show top features for each component
for i in range(5):
    print(f"\n{'='*80}")
    print(f"PC{i+1} - {variance_explained[i]*100:.2f}% variance")
    print(f"{'='*80}")
    
    component_loadings = loadings[f'PC{i+1}'].abs().sort_values(ascending=False)
    
    print("\nTop 10 Features:")
    for feature in component_loadings.head(10).index:
        loading = loadings.loc[feature, f'PC{i+1}']
        print(f"  {feature:30s} {loading:7.3f}")

print("\n✅ PCA interpretation complete")

---

## Logistic regression what predicts performance?

In [ ]:
# Logistic regression: What predicts high performance (>7.0)?
print("="*80)
print("LOGISTIC REGRESSION: PREDICTING HIGH PERFORMERS (>7.0)")
print("="*80)

# Prepare data
X_log = df[predictor_features].dropna()
y_log = df.loc[X_log.index, 'high_performer']

print(f"Sample size: n={len(X_log):,}")
print(f"High performers: {y_log.sum()} ({y_log.mean()*100:.1f}%)")

# Train/test split
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(
    X_log, y_log, test_size=0.2, random_state=42, stratify=y_log
)

# Standardize
scaler_log = StandardScaler()
X_train_scaled = scaler_log.fit_transform(X_train_log)
X_test_scaled = scaler_log.transform(X_test_log)

# Fit logistic regression
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train_scaled, y_train_log)

# Predictions
y_pred_log = logreg.predict(X_test_scaled)
y_pred_proba = logreg.predict_proba(X_test_scaled)[:, 1]

# Performance metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test_log, y_pred_log)
precision = precision_score(y_test_log, y_pred_log)
recall = recall_score(y_test_log, y_pred_log)
f1 = f1_score(y_test_log, y_pred_log)
auc = roc_auc_score(y_test_log, y_pred_proba)

print(f"\nModel Performance:")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1-Score:  {f1:.4f}")
print(f"  AUC-ROC:   {auc:.4f}")

# Feature importance (odds ratios)
odds_ratios = np.exp(logreg.coef_[0])
feature_importance_log = pd.DataFrame({
    'Feature': predictor_features,
    'Coefficient': logreg.coef_[0],
    'Odds_Ratio': odds_ratios
}).sort_values('Coefficient', key=abs, ascending=False)

print("\nTop 15 Features (by |coefficient|):")
print(feature_importance_log.head(15).to_string(index=False))

print("\n✅ Logistic regression complete")

---

## Random Forest

In [ ]:
# Random Forest for high performer prediction
print("="*80)
print("RANDOM FOREST: PREDICTING HIGH PERFORMERS")
print("="*80)

# Use same train/test split
rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf.fit(X_train_log, y_train_log)

# Predictions
y_pred_rf = rf.predict(X_test_log)
y_pred_proba_rf = rf.predict_proba(X_test_log)[:, 1]

# Performance
accuracy_rf = accuracy_score(y_test_log, y_pred_rf)
precision_rf = precision_score(y_test_log, y_pred_rf)
recall_rf = recall_score(y_test_log, y_pred_rf)
f1_rf = f1_score(y_test_log, y_pred_rf)
auc_rf = roc_auc_score(y_test_log, y_pred_proba_rf)

print(f"\nRandom Forest Performance:")
print(f"  Accuracy:  {accuracy_rf:.4f}")
print(f"  Precision: {precision_rf:.4f}")
print(f"  Recall:    {recall_rf:.4f}")
print(f"  F1-Score:  {f1_rf:.4f}")
print(f"  AUC-ROC:   {auc_rf:.4f}")

# Feature importance
feature_importance_rf = pd.DataFrame({
    'Feature': predictor_features,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop 20 Most Important Features:")
print(feature_importance_rf.head(20).to_string(index=False))

# Visualize top 15
plt.figure(figsize=(10, 8))
top_15_features = feature_importance_rf.head(15)
plt.barh(range(len(top_15_features)), top_15_features['Importance'], alpha=0.7)
plt.yticks(range(len(top_15_features)), top_15_features['Feature'])
plt.xlabel('Feature Importance')
plt.title('Random Forest: Top 15 Feature Importances', fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\n✅ Random Forest complete")

---

## Statistical Hypothesis testing

In [ ]:
# Statistical hypothesis testing
print("="*80)
print("STATISTICAL HYPOTHESIS TESTS")
print("="*80)

# Test 1: Gender differences in ratings
print("\n" + "="*80)
print("TEST 1: GENDER DIFFERENCES IN RATINGS")
print("="*80)

male_ratings = df[df['username_gender_hint'] == 'male']['Rating'].dropna()
female_ratings = df[df['username_gender_hint'] == 'female']['Rating'].dropna()

print(f"Male reviewers: n={len(male_ratings)}, mean={male_ratings.mean():.2f}, SD={male_ratings.std():.2f}")
print(f"Female reviewers: n={len(female_ratings)}, mean={female_ratings.mean():.2f}, SD={female_ratings.std():.2f}")

# Independent t-test
t_stat, p_value = ttest_ind(male_ratings, female_ratings)
cohens_d = (male_ratings.mean() - female_ratings.mean()) / np.sqrt(
    ((len(male_ratings)-1)*male_ratings.std()**2 + (len(female_ratings)-1)*female_ratings.std()**2) / 
    (len(male_ratings) + len(female_ratings) - 2)
)

print(f"\nIndependent t-test:")
print(f"  t({len(male_ratings)+len(female_ratings)-2}) = {t_stat:.4f}")
print(f"  p = {p_value:.4f} {'***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'}")
print(f"  Cohen's d = {cohens_d:.4f}")
print(f"  Effect size: {'negligible' if abs(cohens_d) < 0.2 else 'small' if abs(cohens_d) < 0.5 else 'medium' if abs(cohens_d) < 0.8 else 'large'}")

# Test 2: Temporal trends - do ratings change over time windows?
print("\n" + "="*80)
print("TEST 2: RATINGS BY REVIEW WINDOW (ANOVA)")
print("="*80)

window_groups = []
window_labels = []
for window in df['review_window'].dropna().unique():
    window_ratings = df[df['review_window'] == window]['Rating'].dropna()
    if len(window_ratings) > 0:
        window_groups.append(window_ratings)
        window_labels.append(window)
        print(f"{window}: n={len(window_ratings)}, mean={window_ratings.mean():.2f}, SD={window_ratings.std():.2f}")

# One-way ANOVA
f_stat, p_value_anova = f_oneway(*window_groups)
print(f"\nOne-way ANOVA:")
print(f"  F({len(window_groups)-1}, {sum(len(g) for g in window_groups)-len(window_groups)}) = {f_stat:.4f}")
print(f"  p = {p_value_anova:.4e} {'***' if p_value_anova < 0.001 else '**' if p_value_anova < 0.01 else '*' if p_value_anova < 0.05 else 'ns'}")

# Test 3: Sentiment vs Rating correlation by movie
print("\n" + "="*80)
print("TEST 3: SENTIMENT-RATING CORRELATION BY MOVIE")
print("="*80)

for movie in df['Movie_Title'].unique()[:5]:  # First 5 movies
    movie_data = df[df['Movie_Title'] == movie]
    if len(movie_data) > 30:
        r, p = pearsonr(movie_data['vader_compound'].dropna(), 
                       movie_data.loc[movie_data['vader_compound'].notna(), 'Rating'])
        print(f"{movie:30s} r={r:.3f}, p={p:.4e}, n={len(movie_data)}")

print("\n✅ Statistical tests complete")

---

## Actionable Insights

In [ ]:
# Generate actionable insights for "Roots" positioning
print("="*80)
print("ACTIONABLE INSIGHTS FOR 'ROOTS' POSITIONING")
print("="*80)

# Key findings summary
print("\n" + "="*80)
print("1. RATING PREDICTORS (R² = 0.26)")
print("="*80)
print("✓ Positive sentiment (vader_compound) is THE strongest predictor (r=0.264)")
print("✓ Complex vocabulary increases ratings (syllable_count OR=2.13)")
print("✓ Fear emotion is POSITIVE for folk horror (OR=1.32)")
print("✓ Hate statements tank ratings (r=-0.141)")
print("✓ Questions/wishes indicate dissatisfaction (both negative)")

print("\n" + "="*80)
print("2. AUDIENCE SEGMENTS (3 CLUSTERS)")
print("="*80)
print("✓ The Lovers (52%): Rating 6.89, positive sentiment (+0.699)")
print("✓ The Conflicted Critics (44%): Rating 6.07, negative sentiment (-0.404)")
print("✓ The Haters (4%): Rating 4.47, hate statements (1.10 per review)")
print("\n→ TARGET: The Lovers (largest, highest ratings)")
print("→ CONVERT: The Conflicted Critics (reachable with right messaging)")
print("→ AVOID: Don't trigger The Haters (disgust, misleading marketing)")

print("\n" + "="*80)
print("3. TEMPORAL DYNAMICS (F=48.82, p<0.001)")
print("="*80)
print("✓ 5+ Years: 7.48 rating (HIGHEST - legacy effect)")
print("✓ Years 4-5: 7.23 rating (positive momentum)")
print("✓ Opening Year: 6.06 rating (mixed initial reception)")
print("✓ Years 2-3: 5.94-6.10 (LOWEST - backlash period)")
print("\n→ STRATEGY: Position for long-term legacy, not just opening weekend")
print("→ EXPECTATION: Initial reviews will be mixed - normal pattern")
print("→ PAYOFF: Films gain appreciation over 4-5 years")

print("\n" + "="*80)
print("4. LATENT FACTORS (PCA - 5 COMPONENTS)")
print("="*80)
print("✓ PC1 (13%): Review depth - detailed analysis vs brief reactions")
print("✓ PC2 (9%): Linguistic sophistication - academic vs casual")
print("✓ PC3 (7%): Negative engagement - controversial negativity")
print("✓ PC4 (6%): Polarization - divisive debates")
print("✓ PC5 (5%): Emotional accessibility - enthusiastic vs measured")
print("\n→ INSIGHT: Folk horror generates DIVERSE response styles")
print("→ MARKETING: Appeal to both sophisticated analysts AND enthusiasts")

print("\n" + "="*80)
print("5. CRITICAL SUCCESS FACTORS (Random Forest AUC=0.80)")
print("="*80)
print("Top predictors of high performance (>7.0 rating):")
print("  1. Movie release year (temporal effect)")
print("  2. Years since release (legacy building)")
print("  3. Positive sentiment")
print("  4. Compound sentiment")
print("  5. Noun ratio (substantive language)")
print("\n→ POSITIONING: Build anticipation for a film that will GROW in stature")

print("\n" + "="*80)
print("6. GENDER EFFECTS (MINIMAL)")
print("="*80)
print("✓ Males: 6.53 rating (n=2,146)")
print("✓ Females: 6.15 rating (n=444)")
print("✓ Difference: p=0.007 but Cohen's d=0.14 (negligible)")
print("\n→ MARKETING: Don't over-gender the campaign - appeal is universal")

print("\n" + "="*80)
print("FINAL RECOMMENDATION FOR 'ROOTS'")
print("="*80)
print("""
POSITIONING STRATEGY:
- Frame as a LEGACY folk horror film, not just a theatrical release
- Emphasize EMOTIONAL DEPTH (fear + anticipation, avoid disgust)
- Target sophisticated horror fans who write detailed reviews
- Prepare for mixed opening-year reception (normal pattern)
- Build long-term festival/streaming strategy (5+ year appreciation)

MARKETING GUARDRAILS:
- Avoid misleading trailers that trigger hate/disgust
- Don't overpromise - let quality speak for itself
- Minimize question-inducing marketing (clarity > mystery overload)
- Generate positive sentiment early with influencer screenings

AUDIENCE TARGETING:
- Primary: "The Lovers" (52% - positive, enthusiastic, trust-based)
- Secondary: "Conflicted Critics" (44% - sophisticated, reachable)
- Avoid: Triggering "The Haters" (4% - vocal but small)

SUCCESS METRICS:
- Opening year: Target 6.0+ rating (on par with successful folk horror)
- Year 5+: Target 7.5+ rating (legacy appreciation)
- Sentiment: Maintain vader_compound > 0.3 (positive territory)
- Minimize hate_count < 0.15 per review
""")

print("\n✅ Analysis complete - ready for investor presentation")